# Day 5: Matplotlib/Seaborn基礎（データ可視化）- ベーシック版

## 学習目標
- Matplotlibの基本的な使い方とオブジェクト指向インターフェース
- Seabornを使った統計的可視化
- 複雑なレイアウトとカスタマイズ
- インタラクティブな可視化の基礎

## データ可視化の重要性
- EDA（探索的データ分析）における中心的な役割
- パターン、異常値、相関関係の発見
- ステークホルダーへの効果的なコミュニケーション

## 1. 環境設定とライブラリのインポート

In [ ]:
# 基本ライブラリ
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import platform

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec
import seaborn as sns

# 警告の抑制
import warnings
warnings.filterwarnings('ignore')

# スタイル設定
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# MacOS用の日本語フォント設定
if platform.system() == 'Darwin':  # macOS
    from matplotlib import rcParams
    
    # 日本語フォントを設定
    rcParams['font.sans-serif'] = ['Hiragino Sans', 'Hiragino Kaku Gothic ProN', 
                                   'Hiragino Maru Gothic Pro', 'Arial Unicode MS']
    rcParams['font.family'] = 'sans-serif'
    
    # 追加の設定
    rcParams['pdf.fonttype'] = 42  # PDFでのフォント埋め込み
    rcParams['ps.fonttype'] = 42   # PostScriptでのフォント埋め込み
    
    print("日本語フォント設定完了: Hiragino Sans を使用")
    
else:
    # Windows/Linux用の設定
    rcParams['font.sans-serif'] = ['Yu Gothic', 'Meiryo', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']

# マイナス記号の表示設定
rcParams['axes.unicode_minus'] = False

# その他の設定
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.max_open_warning'] = 50

print(f"\nMatplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")

## 2. サンプルデータの生成

In [ ]:
# シード設定
np.random.seed(42)

# 時系列データ（1年分の売上データ）
dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
n_days = len(dates)

# トレンドとシーズナリティを含む売上データ
trend = np.linspace(1000, 1500, n_days)
seasonal = 200 * np.sin(2 * np.pi * np.arange(n_days) / 365.25)
noise = np.random.normal(0, 50, n_days)
sales = trend + seasonal + noise

# カテゴリ別売上
categories = ['Electronics', 'Clothing', 'Food', 'Books', 'Sports']
category_sales = np.random.multinomial(sales.astype(int), 
                                     [0.3, 0.25, 0.2, 0.15, 0.1])

# DataFrameの作成
df = pd.DataFrame({
    'date': dates,
    'total_sales': sales,
    'Electronics': category_sales[:, 0],
    'Clothing': category_sales[:, 1],
    'Food': category_sales[:, 2],
    'Books': category_sales[:, 3],
    'Sports': category_sales[:, 4],
    'temperature': 20 + 10 * np.sin(2 * np.pi * np.arange(n_days) / 365.25) + 
                  np.random.normal(0, 2, n_days),
    'visitors': np.random.poisson(1000 + 300 * np.sin(2 * np.pi * np.arange(n_days) / 365.25), n_days),
    'conversion_rate': np.clip(0.02 + 0.01 * np.sin(2 * np.pi * np.arange(n_days) / 365.25) + 
                              np.random.normal(0, 0.005, n_days), 0.01, 0.05)
})

# 月次集計データ
df['month'] = df['date'].dt.to_period('M')
monthly_df = df.groupby('month').agg({
    'total_sales': 'sum',
    'Electronics': 'sum',
    'Clothing': 'sum',
    'Food': 'sum',
    'Books': 'sum',
    'Sports': 'sum',
    'visitors': 'sum',
    'conversion_rate': 'mean'
}).reset_index()

print("データの概要:")
print(df.head())
print(f"\nデータの形状: {df.shape}")
print(f"\n基本統計量:")
print(df[['total_sales', 'temperature', 'visitors', 'conversion_rate']].describe())

## 3. Matplotlibの基本

### 3.1 Figure と Axes の概念

In [ ]:
# オブジェクト指向インターフェースの基本
fig, ax = plt.subplots(figsize=(12, 6))

# 時系列プロット
ax.plot(df['date'], df['total_sales'], label='Total Sales', linewidth=2, alpha=0.8)

# 移動平均の追加
rolling_mean = df['total_sales'].rolling(window=30, center=True).mean()
ax.plot(df['date'], rolling_mean, label='30-day Moving Average', 
        color='red', linewidth=3, alpha=0.9)

# グラフの装飾
ax.set_title('Daily Sales Trend with Moving Average', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Sales ($)', fontsize=12)

# x軸の日付フォーマット
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.xaxis.set_major_locator(mdates.MonthLocator())
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# グリッドとレジェンド
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(loc='upper left', frameon=True, shadow=True)

# 領域の塗りつぶし（標準偏差）
rolling_std = df['total_sales'].rolling(window=30, center=True).std()
ax.fill_between(df['date'], 
               rolling_mean - rolling_std, 
               rolling_mean + rolling_std, 
               alpha=0.2, color='red', label='±1 STD')

plt.tight_layout()
plt.show()

### 3.2 複数のサブプロット

In [ ]:
# GridSpecを使った複雑なレイアウト
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)

# メインプロット（上部全体）
ax1 = fig.add_subplot(gs[0, :])
for category in categories:
    ax1.plot(df['date'], df[category], label=category, alpha=0.7)
ax1.set_title('Sales by Category Over Time', fontsize=14, fontweight='bold')
ax1.set_ylabel('Sales ($)')
ax1.legend(loc='upper left', ncol=5)
ax1.grid(True, alpha=0.3)

# カテゴリ別月次売上（左下）
ax2 = fig.add_subplot(gs[1, :2])
monthly_df.set_index('month')[categories].plot(kind='bar', stacked=True, ax=ax2)
ax2.set_title('Monthly Sales by Category (Stacked)', fontsize=12)
ax2.set_xlabel('Month')
ax2.set_ylabel('Sales ($)')
ax2.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)

# 相関ヒートマップ（右下）
ax3 = fig.add_subplot(gs[1:, 2])
corr_data = df[['total_sales', 'temperature', 'visitors', 'conversion_rate']].corr()
sns.heatmap(corr_data, annot=True, cmap='coolwarm', center=0, 
           square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax3)
ax3.set_title('Correlation Matrix', fontsize=12)

# 散布図（左下の下）
ax4 = fig.add_subplot(gs[2, 0])
scatter = ax4.scatter(df['temperature'], df['total_sales'], 
                     c=df['conversion_rate'], s=30, alpha=0.6, cmap='viridis')
ax4.set_xlabel('Temperature (°C)')
ax4.set_ylabel('Sales ($)')
ax4.set_title('Sales vs Temperature', fontsize=12)
cbar = plt.colorbar(scatter, ax=ax4)
cbar.set_label('Conversion Rate', rotation=270, labelpad=15)

# 箱ひげ図（中下）
ax5 = fig.add_subplot(gs[2, 1])
df_melt = df[categories].melt(var_name='Category', value_name='Sales')
sns.boxplot(data=df_melt, x='Category', y='Sales', ax=ax5)
ax5.set_title('Sales Distribution by Category', fontsize=12)
plt.setp(ax5.xaxis.get_majorticklabels(), rotation=45)

plt.suptitle('Comprehensive Sales Dashboard', fontsize=18, fontweight='bold', y=0.98)
plt.show()

## 4. Seabornの高度な機能

### 4.1 統計的可視化

In [ ]:
# ペアプロット
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. 回帰プロット
sns.regplot(data=df, x='visitors', y='total_sales', ax=axes[0, 0], 
           scatter_kws={'alpha': 0.5}, line_kws={'color': 'red'})
axes[0, 0].set_title('Sales vs Visitors with Regression Line')

# 2. バイオリンプロット
df['quarter'] = df['date'].dt.quarter
sns.violinplot(data=df, x='quarter', y='total_sales', ax=axes[0, 1])
axes[0, 1].set_title('Sales Distribution by Quarter')
axes[0, 1].set_xlabel('Quarter')

# 3. スウォームプロット + ボックスプロット
df['weekday'] = df['date'].dt.day_name()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sample_df = df.sample(n=100, random_state=42)  # サンプリングして点を減らす

sns.boxplot(data=df, x='weekday', y='conversion_rate', 
           order=weekday_order, ax=axes[1, 0], color='lightgray')
sns.swarmplot(data=sample_df, x='weekday', y='conversion_rate', 
             order=weekday_order, ax=axes[1, 0], size=3, alpha=0.7)
axes[1, 0].set_title('Conversion Rate by Day of Week')
plt.setp(axes[1, 0].xaxis.get_majorticklabels(), rotation=45)

# 4. KDEプロット（カーネル密度推定）
for category in categories[:3]:  # 上位3カテゴリのみ
    sns.kdeplot(data=df, x=category, ax=axes[1, 1], label=category, fill=True, alpha=0.3)
axes[1, 1].set_title('Sales Distribution by Top 3 Categories')
axes[1, 1].set_xlabel('Sales ($)')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

### 4.2 FacetGridを使った多次元可視化

In [ ]:
# データの準備（長形式に変換）
df_long = df[['date', 'Electronics', 'Clothing', 'Food']].melt(
    id_vars=['date'], var_name='Category', value_name='Sales'
)
df_long['month'] = df_long['date'].dt.month
df_long['quarter'] = df_long['date'].dt.quarter

# FacetGrid
g = sns.FacetGrid(df_long, col='Category', height=4, aspect=1.2)
g.map(sns.histplot, 'Sales', bins=30, kde=True)
g.add_legend()
g.fig.suptitle('Sales Distribution by Category', fontsize=16, y=1.02)
plt.show()

# カテゴリ別の月次トレンド
monthly_long = df_long.groupby(['month', 'Category'])['Sales'].sum().reset_index()

plt.figure(figsize=(14, 6))
sns.lineplot(data=monthly_long, x='month', y='Sales', hue='Category', 
            style='Category', markers=True, dashes=False, linewidth=2.5)
plt.title('Monthly Sales Trend by Category', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Sales ($)')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. 高度なカスタマイズテクニック

### 5.1 カスタムカラーマップとスタイル

In [ ]:
# カスタムカラーパレットの作成
custom_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#F7DC6F', '#BB8FCE']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 円グラフ with カスタムスタイル
category_totals = df[categories].sum().sort_values(ascending=False)
explode = [0.05 if i == 0 else 0 for i in range(len(categories))]

wedges, texts, autotexts = ax1.pie(category_totals, labels=category_totals.index, 
                                   autopct='%1.1f%%', startangle=90, 
                                   colors=custom_colors, explode=explode,
                                   shadow=True, textprops={'fontsize': 12})

# 自動テキストのスタイリング
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

ax1.set_title('Total Sales Distribution by Category', fontsize=14, fontweight='bold', pad=20)

# 極座標プロット（レーダーチャート）
# 各カテゴリの月平均売上を正規化
monthly_avg = df[categories].resample('M', on='date').mean()
normalized_avg = (monthly_avg - monthly_avg.min()) / (monthly_avg.max() - monthly_avg.min())

angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]  # 閉じるため

ax2 = plt.subplot(122, projection='polar')
for idx, month in enumerate(['Jan', 'Jun', 'Dec']):
    month_idx = [0, 5, 11][idx]
    if month_idx < len(normalized_avg):
        values = normalized_avg.iloc[month_idx].tolist()
        values += values[:1]
        ax2.plot(angles, values, 'o-', linewidth=2, label=month)
        ax2.fill(angles, values, alpha=0.15)

ax2.set_xticks(angles[:-1])
ax2.set_xticklabels(categories)
ax2.set_ylim(0, 1)
ax2.set_title('Normalized Sales Performance by Category', fontsize=14, fontweight='bold', pad=30)
ax2.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
ax2.grid(True)

plt.tight_layout()
plt.show()

### 5.2 アニメーションと3Dプロット

In [ ]:
# 3Dプロット
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14, 6))

# 3D散布図
ax1 = fig.add_subplot(121, projection='3d')

# データの準備
sample_3d = df.sample(n=200, random_state=42)
x = sample_3d['temperature']
y = sample_3d['visitors']
z = sample_3d['total_sales']
c = sample_3d['conversion_rate']

scatter = ax1.scatter(x, y, z, c=c, cmap='plasma', s=50, alpha=0.6)
ax1.set_xlabel('Temperature (°C)')
ax1.set_ylabel('Visitors')
ax1.set_zlabel('Sales ($)')
ax1.set_title('3D Relationship: Temperature, Visitors, and Sales')

# カラーバー
cbar = plt.colorbar(scatter, ax=ax1, pad=0.1)
cbar.set_label('Conversion Rate', rotation=270, labelpad=15)

# 3D表面プロット
ax2 = fig.add_subplot(122, projection='3d')

# メッシュグリッドの作成
temp_range = np.linspace(x.min(), x.max(), 20)
visitor_range = np.linspace(y.min(), y.max(), 20)
temp_grid, visitor_grid = np.meshgrid(temp_range, visitor_range)

# 簡単な線形モデルで予測（実際にはもっと複雑なモデルを使用）
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(sample_3d[['temperature', 'visitors']], sample_3d['total_sales'])
sales_grid = model.predict(np.c_[temp_grid.ravel(), visitor_grid.ravel()]).reshape(temp_grid.shape)

# 表面プロット
surf = ax2.plot_surface(temp_grid, visitor_grid, sales_grid, cmap='viridis', alpha=0.8)
ax2.set_xlabel('Temperature (°C)')
ax2.set_ylabel('Visitors')
ax2.set_zlabel('Predicted Sales ($)')
ax2.set_title('Sales Prediction Surface')

plt.tight_layout()
plt.show()

## 6. インタラクティブな可視化（Plotly）

In [ ]:
# Plotlyのインストールが必要な場合はコメントを外してください
# !pip install plotly

try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    # インタラクティブな時系列プロット
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=('Daily Sales with Range Slider', 'Category Breakdown'),
        row_heights=[0.7, 0.3]
    )
    
    # メインの売上トレンド
    fig.add_trace(
        go.Scatter(x=df['date'], y=df['total_sales'], 
                  mode='lines', name='Total Sales',
                  line=dict(color='blue', width=2)),
        row=1, col=1
    )
    
    # カテゴリ別売上（積み上げエリアチャート）
    for category in categories:
        fig.add_trace(
            go.Scatter(x=df['date'], y=df[category],
                      mode='lines', name=category,
                      stackgroup='one'),
            row=2, col=1
        )
    
    # レイアウトの更新
    fig.update_xaxes(rangeslider_visible=True, row=2, col=1)
    fig.update_layout(
        height=600,
        title_text="Interactive Sales Dashboard",
        showlegend=True,
        hovermode='x unified'
    )
    
    fig.show()
    
except ImportError:
    print("Plotlyがインストールされていません。")
    print("インタラクティブなグラフを表示するには、'pip install plotly'を実行してください。")

## 7. 実践演習

### 演習1: カスタムダッシュボードの作成

In [ ]:
# TODO: 以下の要件を満たすダッシュボードを作成してください
# 1. 4つのサブプロット (2x2)
# 2. 左上: 月別の売上と訪問者数の二軸グラフ
# 3. 右上: カテゴリ別の売上割合の推移（エリアチャート）
# 4. 左下: 温度と変換率の関係（回帰線付き散布図）
# 5. 右下: 四半期別のカテゴリ売上ヒートマップ

# ヒント:
# - ax.twinx()で二軸グラフを作成
# - df.pivot_table()でヒートマップ用のデータを整形
# - sns.regplot()で回帰線付き散布図

# ここにコードを書いてください
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 実装をここに...


### 演習2: カスタム可視化関数の作成

In [ ]:
# TODO: 時系列データの異常値を視覚化する関数を作成してください
def plot_anomalies(data, column, window=30, n_std=2):
    """
    時系列データの異常値を検出して視覚化する
    
    Parameters:
    -----------
    data : DataFrame
        時系列データを含むDataFrame
    column : str
        分析対象のカラム名
    window : int
        移動平均のウィンドウサイズ
    n_std : float
        異常値とみなす標準偏差の倍数
    
    Returns:
    --------
    fig, ax : matplotlib objects
    """
    # ここに実装を書いてください
    pass

# 関数のテスト
# plot_anomalies(df, 'total_sales', window=30, n_std=2)

## 8. ベストプラクティスとまとめ

### 効果的なデータ可視化のガイドライン

1. **目的に応じた適切なグラフタイプの選択**
   - 時系列 → 折れ線グラフ
   - カテゴリ比較 → 棒グラフ
   - 分布 → ヒストグラム、箱ひげ図
   - 相関 → 散布図、ヒートマップ
   - 部分と全体 → 円グラフ、ツリーマップ

2. **色の効果的な使用**
   - カテゴリカルデータ：異なる色相
   - 連続データ：グラデーション
   - 色覚異常に配慮した配色

3. **情報の階層化**
   - 最も重要な情報を目立たせる
   - 補助的な情報は控えめに

4. **インタラクティビティの活用**
   - 大量のデータを扱う場合
   - 探索的分析

5. **再現性の確保**
   - ランダムシードの設定
   - バージョン管理
   - コードのモジュール化

### 解答例

In [ ]:
# 演習1の解答例
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. 月別の売上と訪問者数の二軸グラフ
ax1 = axes[0, 0]
color1 = 'tab:blue'
ax1.bar(monthly_df['month'].astype(str), monthly_df['total_sales'], 
        color=color1, alpha=0.7, label='Sales')
ax1.set_xlabel('Month')
ax1.set_ylabel('Sales ($)', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
ax1.set_title('Monthly Sales and Visitors', fontweight='bold')

ax1_twin = ax1.twinx()
color2 = 'tab:red'
ax1_twin.plot(monthly_df['month'].astype(str), monthly_df['visitors'], 
             color=color2, marker='o', linewidth=2, label='Visitors')
ax1_twin.set_ylabel('Visitors', color=color2)
ax1_twin.tick_params(axis='y', labelcolor=color2)

# 2. カテゴリ別の売上割合の推移
ax2 = axes[0, 1]
monthly_pct = monthly_df[categories].div(monthly_df[categories].sum(axis=1), axis=0) * 100
monthly_pct.plot(kind='area', stacked=True, ax=ax2, alpha=0.7)
ax2.set_title('Category Sales Percentage Over Time', fontweight='bold')
ax2.set_ylabel('Percentage (%)')
ax2.set_xlabel('Month')
ax2.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')

# 3. 温度と変換率の関係
ax3 = axes[1, 0]
sns.regplot(data=df, x='temperature', y='conversion_rate', ax=ax3,
           scatter_kws={'alpha': 0.5}, line_kws={'color': 'red', 'linewidth': 2})
ax3.set_title('Temperature vs Conversion Rate', fontweight='bold')
ax3.set_xlabel('Temperature (°C)')
ax3.set_ylabel('Conversion Rate')

# 4. 四半期別のカテゴリ売上ヒートマップ
ax4 = axes[1, 1]
quarterly_sales = df.groupby('quarter')[categories].sum()
sns.heatmap(quarterly_sales.T, annot=True, fmt='.0f', cmap='YlOrRd', ax=ax4)
ax4.set_title('Quarterly Sales Heatmap by Category', fontweight='bold')
ax4.set_xlabel('Quarter')
ax4.set_ylabel('Category')

plt.tight_layout()
plt.show()

In [ ]:
# 演習2の解答例
def plot_anomalies(data, column, window=30, n_std=2):
    """
    時系列データの異常値を検出して視覚化する
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # 移動平均と標準偏差の計算
    rolling_mean = data[column].rolling(window=window, center=True).mean()
    rolling_std = data[column].rolling(window=window, center=True).std()
    
    # 上限と下限の計算
    upper_bound = rolling_mean + n_std * rolling_std
    lower_bound = rolling_mean - n_std * rolling_std
    
    # 異常値の検出
    anomalies = data[(data[column] > upper_bound) | (data[column] < lower_bound)]
    
    # プロット
    ax.plot(data['date'], data[column], label='Original Data', alpha=0.7)
    ax.plot(data['date'], rolling_mean, label=f'{window}-day Moving Average', 
            color='red', linewidth=2)
    
    # 信頼区間
    ax.fill_between(data['date'], lower_bound, upper_bound, 
                   alpha=0.2, color='gray', label=f'±{n_std} STD')
    
    # 異常値をハイライト
    ax.scatter(anomalies['date'], anomalies[column], 
              color='red', s=50, zorder=5, label='Anomalies')
    
    ax.set_title(f'Anomaly Detection in {column}', fontsize=16, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel(column)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 統計情報の表示
    textstr = f'Anomalies detected: {len(anomalies)} ({len(anomalies)/len(data)*100:.1f}%)'
    ax.text(0.02, 0.95, textstr, transform=ax.transAxes, 
           fontsize=12, verticalalignment='top',
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    return fig, ax

# 関数のテスト
plot_anomalies(df, 'total_sales', window=30, n_std=2)
plt.show()

## 完了！

今日はMatplotlibとSeabornを使った高度なデータ可視化技術を学びました。

### 学んだこと：
- Matplotlibのオブジェクト指向インターフェース
- 複雑なレイアウトの作成（GridSpec）
- Seabornによる統計的可視化
- 3Dプロットとインタラクティブな可視化
- カスタム関数による再利用可能な可視化

### 次のステップ：
- Plotly/Dashを使ったインタラクティブダッシュボード
- 地理空間データの可視化（GeoPandas）
- アニメーションの作成
- 実際のビジネスデータでの実践